In [ ]:
#  raw data not available

In [1]:
import os
from parse_soft import ParseSoft
from search_soft import SearchSoft
from enrich_soft import EnrichSoft
from label_data import LabelData

%load_ext autoreload
%autoreload 2

In [2]:
# step 1: set defaults
geo = "GSE161529"
soft_dir = '/home/yuan/data'
meta_dir = '../data'
download_dir = '/home/yuan/rawdata/SRR'

In [3]:
# step 2: parse soft, and enrich

# analyze soft file
parser = ParseSoft(soft_dir)
soft_path = parser.soft_local_path(geo)
print('Retrieve meta data from soft file: ', soft_path)
softer = SearchSoft(soft_path)
data = softer.filter_data(softer.parse_rows)

# save to json
LabelData(meta_dir).save(data)

Retrieve meta data from soft file:  /home/yuan/data/ftp.ncbi.nlm.nih.gov/geo/series/GSE161nnn/GSE161529/soft/GSE161529_family.soft.gz
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE161nnn/GSE161529/metadata.json


In [4]:
# step 3a: enrich data
data = LabelData(meta_dir).get_meta(geo)
enriched = EnrichSoft(data)()
LabelData(meta_dir).save(enriched)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE161nnn/GSE161529/metadata.json


In [15]:
# step 3b: label samples manually
'''
'''
from label_sample import LabelSample

def collator(sample):
    c = LabelSample(sample)
    c._protocol('scrna-seq', "10x3")
    c._update('tissue', c.tissue + ', breast')
    c._update('group', 'patient')
    
    # cell_type
    if 'Normal Total cells' in c.title:
        c._update('cell_type', 'normal total cells')
        c._update('tissue', 'normal tissue, breast')
    elif 'Normal Epithelial cells' in c.title:
        c._update('cell_type', 'epithelial cells')
        c._update('tissue', 'normal tissue, breast')
    elif 'pre-neoplastic Total cells' in c.title:
        c._update('cell_type', 'preneoplastic total cells')
        c._update('tissue', 'preneoplastic tissue, breast')
    elif 'tumour Total cells' in c.title:
        c._update('cell_type', 'tumor total cells')
        c._update('tissue', 'tumor tissue, breast')
    elif 'tumour Lymph-node cells' in c.title:
        c._update('cell_type', 'lymph-node cells')
        c._update('tissue', 'tumor tissue, breast')
    
    # disease
    if c.title.startswith('BRCA1'):
        c._update('disease', 'BRCA1 breast cancer')
    elif 'Triple negative tumor' in c.title:
        c._update('disease', 'triple-negative breast cancer')
    elif 'Triple negative BRCA1 tumour' in c.title:
        c._update('disease', 'BRCA1 and Triple-negative breast cancer')
    elif 'HER2+ tumour' in c.title:
        c._update('disease', 'HER2 breast cancer')
    elif 'PR+ tumour' in c.title:
        c._update('disease', 'PR breast cancer')
    elif 'ER+ tumour' in c.title:
        c._update('disease', 'ER breast cancer')
    else:
        c._update('disease', 'breast cancer')

data = LabelData(meta_dir).get_meta(geo)
for sample_id, sample in data['samples'].items():
    collator(sample)
LabelData(meta_dir).save(data)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE161nnn/GSE161529/metadata.json


In [16]:
for sample_id, sample in data['samples'].items():
    # print(sample['characteristics'])
    print(sample['sample_title'], sample['labels'])

Normal Total cells from Patient 0092 {'tissue': 'normal tissue, breast', 'group': 'patient', 'cell_type': 'normal total cells', 'disease': 'breast cancer'}
Normal Total cells from Patient 0019 {'tissue': 'normal tissue, breast', 'group': 'patient', 'cell_type': 'normal total cells', 'disease': 'breast cancer'}
Normal Epithelial cells from Patient 0280 {'tissue': 'normal tissue, breast', 'group': 'patient', 'cell_type': 'epithelial cells', 'disease': 'breast cancer'}
Normal Epithelial cells from Patient 0093 {'tissue': 'normal tissue, breast', 'group': 'patient', 'cell_type': 'epithelial cells', 'disease': 'breast cancer'}
Normal Total cells from Patient 0093 {'tissue': 'normal tissue, breast', 'group': 'patient', 'cell_type': 'normal total cells', 'disease': 'breast cancer'}
Normal Epithelial cells from Patient 1469 {'tissue': 'normal tissue, breast', 'group': 'patient', 'cell_type': 'epithelial cells', 'disease': 'breast cancer'}
Normal Epithelial cells from Patient 0408 {'tissue': 'n

In [17]:
# step 4a: load some data
from utils import Utils

# data are determined previously
samn_srr = Utils.from_json('../results/samn_srr.json')
print(len(samn_srr))

47


In [18]:
# step 4b: Given BioSample, parse SRR accessions into samples
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
enriched = ParseSra.parse_srr(data, samn_srr)
LabelData(meta_dir).save(enriched)

biosamples = 69, SRR accessions = 0.
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE161nnn/GSE161529/metadata.json


In [19]:
# step 5: merge srr_urls into data

# sample.<sample_id>.SRR.<SRR accession>."ftp.sra.ebi.ac.uk"
file_name = 'srr_fastq_urls_simple.json'
urls_json = os.path.join('../results', file_name)
urls = Utils.from_json(urls_json)

# parse urls
data = LabelData(meta_dir).get_meta(geo)
enriched = ParseSra.parse_srr_urls(data, urls)
LabelData(meta_dir).save(enriched)

{'srr': 69, 'available': 0, 'unknown': 0, 'updated': 0}
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE161nnn/GSE161529/metadata.json


In [20]:
# step 6a: download raw data using biosample accession using nextflow
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_biosample(sample_iter)
# run the bash script showed here
# Outcome: nf download failed

Number of biosamples:  69
Number of biosamples with SRR:  0
bash /home/yuan/rawdata/SRR/GSE161529/fetch1.sh


In [21]:
#Step 6b: collect all SRR without local_fastq then execute bash run2.sh
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_ebi_srr(sample_iter)
# run the bash script showed here

Number of SRR:  0
bash /home/yuan/rawdata/SRR/GSE161529/fetch2.sh


In [22]:
#Step 6c: collect all SRR without local_fastq then execute bash run3.sh
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_srr(sample_iter)
# run the bash script showed here

Number of SRR:  0
bash /home/yuan/rawdata/SRR/GSE161529/fetch3.sh


In [23]:
# step 7: parse path of local fastq.gz with SRR in GEO data
# Note: run the process multiple times to make sure all SRRs are accessed
# print all samples if not any local *.fastq.gz is parsed
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
data, unparsing = ParseSra.parse_local_fastq(data, download_dir)
if unparsing:
    print(geo, len(unparsing), unparsing[:6])
LabelData(meta_dir).save(data)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE161nnn/GSE161529/metadata.json
